# Modelado de Datos

En esta fase del proyecto se prepara el conjunto de datos para el entrenamiento de modelos predictivos.
Se parte del análisis exploratorio realizado previamente (EDA) y se construye el entorno de modelado,
incluyendo limpieza, transformación de variables y partición de los datos.


In [1]:
import pandas as pd
import sys

from sklearn.model_selection import train_test_split


In [2]:
import sys
sys.path.append("../src")

from data_preparation import preparar_datos_modelo

In [3]:
df = pd.read_csv("../data/raw/Cobertura_móvil_por_tecnología,_departamento_y_municipio_por_proveedor_20251208.csv")
df.head()


,AÑO,TRIMESTRE,PROVEEDOR,COD DEPARTAMENTO,DEPARTAMENTO,COD MUNICIPIO,MUNICIPIO,CABECERA MUNICIPAL,COD CENTRO POBLADO,CENTRO POBLADO,COBERTURA 2G,COBERTURA 3G,"COBERTURA HSPA+, HSPA+DC",COBERTUTA 4G,COBERTURA LTE,COBERTURA 5G
0,2023,3,COLOMBIA MOVIL S.A ESP,27,CHOCÓ,27250.0,EL LITORAL DEL SAN JUAN,N,27250034.0,TORDÓ,N,N,N,S,N,N
1,2023,3,COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,5,ANTIOQUIA,5495.0,NECHÍ,N,5495003.0,LA CONCHA,S,S,S,S,N,N
2,2022,3,COLOMBIA MOVIL S.A ESP,70,SUCRE,70508.0,OVEJAS,N,70508006.0,DON GABRIEL,N,N,N,S,N,N
3,2021,4,AVANTEL S.A.S,73,TOLIMA,73043.0,ANZOÁTEGUI,S,73043000.0,ANZOÁTEGUI,S,S,S,N,N,N
4,2021,2,COMUNICACION CELULAR S A COMCEL S A,50,META,50150.0,CASTILLA LA NUEVA,N,50150001.0,SAN LORENZO,S,S,S,S,N,N


El dataset utilizado corresponde a información de cobertura móvil por tecnología,
departamento, municipio y proveedor.  
Este conjunto de datos fue previamente analizado en la fase de EDA.


In [4]:
X_train, X_test, y_train, y_test = preparar_datos_modelo(df)


print("Pipeline de modelado funcionando correctamente")


Pipeline de modelado funcionando correctamente


In [5]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("Distribución de la variable objetivo:")
y_train.value_counts()


X_train: (285061, 8168)
X_test: (122169, 8168)
Distribución de la variable objetivo:


COBERTUTA 4G
N    232392
S     52669
Name: count, dtype: int64

Durante la preparación del entorno de modelado se realizaron las siguientes acciones:

- Eliminación de valores nulos
- Codificación de variables categóricas mediante One-Hot Encoding
- Separación de variables predictoras (X) y variable objetivo (y)
- División del conjunto de datos en entrenamiento (70%) y prueba (30%)

Estas acciones garantizan que los datos estén en condiciones adecuadas para el entrenamiento de modelos.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)
